In [1]:
import re
import json
import pandas as pd
from tqdm import tqdm
import numpy as np
from langdetect import detect, DetectorFactory
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
import nltk
import re
import pprint
from collections import Counter
from langdetect import detect, DetectorFactory
import string
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
data = []
with open('../data/yelp-dataset/yelp_academic_dataset_review.json',encoding="utf8") as f:
    for line in f:
        data.append(json.loads(line))

In [3]:
type(data[2])

dict

In [4]:
DF = pd.DataFrame(data)

In [5]:
type(DF['stars'][0])

numpy.float64

In [6]:
DF_drop = DF[DF['stars']!= 3.0]

In [7]:
DF_drop['stars'].value_counts()

5.0    2933082
4.0    1468985
1.0    1002159
2.0     542394
Name: stars, dtype: int64

In [8]:
positive = []

for i in DF_drop.stars:
    if i < 3.0:
        positive.append(0)
    else:
        positive.append(1)
DF_drop['positive'] = positive

C:\Users\Gabo\Anaconda3\envs\Geopandas\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
DF_pos = DF_drop[DF_drop['positive']==1]
DF_neg = DF_drop[DF_drop['positive']==0]

In [10]:
DF_pos_sample = DF_pos.sample(n=50000,random_state=123)
DF_neg_sample = DF_neg.sample(n=50000,random_state=123)

In [11]:
DF_pos_sample['stars'].value_counts()

5.0    33298
4.0    16702
Name: stars, dtype: int64

In [12]:
DF_neg_sample['stars'].value_counts() 

1.0    32553
2.0    17447
Name: stars, dtype: int64

In [13]:
yelp = pd.concat([DF_pos_sample,DF_neg_sample])

In [14]:
yelp.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,positive
4928306,7TWd_t1xe5YQX1f20MdXEg,X20U1YunHVHhqiQe3rkJ-Q,WAIEdzllzVnSwcTE7GEn6A,4.0,1,0,0,I went there because my front bumper fell off ...,2015-07-22 21:14:56,1
7160,XhX-U8N-sY6v5VaaxU2IUg,xR4engcg6Vm8xToBgKI5FA,pVCF9rIPF4y697QeRMpweA,4.0,1,0,0,"Nice, clean, large grocery store with great va...",2014-02-05 23:10:33,1
1675101,O6CDUki8r7yYcQOwO4kY9w,xzdEPPOsLzRiHN9Z3nPKag,9xdVIIzHp12Zrox5aFaaHA,4.0,5,0,1,"Ladies and Gentlemen, we are officially witnes...",2011-08-10 01:30:52,1
5702417,14p07vYeRrEvOt7MrOp-Vg,zWWcik1fRPZviBCQLC26FQ,IZ33QKVggRtHdMdWoDPi6Q,4.0,1,0,0,I think this place deserves a much better rati...,2016-09-23 21:18:34,1
753841,xnRBYPoavUMA_ept3VZ86A,jUZW1alRK8QvdZ-pH1OquQ,cOJ1uIVIHCiefUyWG2wDfw,4.0,8,3,5,Never heard of this place. Really cool spot! ...,2014-05-28 03:04:02,1


In [19]:
yelp.to_csv('../data/yelp-dataset/df_yelp.csv',index=False)

In [20]:
yelp = pd.read_csv('../data/yelp-dataset/df_yelp.csv')

In [21]:
# Code to detect languages and then remove those not labelled as English. This removed about 500 examples but did
# not clean all of the text of non english words. This might not be an issue though but not sure.

yelp.text.isna().value_counts()

count = 0 #check for empty strings
for i in range(len(yelp.text)):
    if not yelp.text[i].strip():
        count +=1
    else:
        continue

print(count)

DetectorFactory.seed = 0
final = []
for i in range(len(yelp.text)):
    try:
        language = detect(yelp.text[i])
    except:
        language = "error"
        print("This row throws an error:", yelp.text[i])
    final.append(language)


yelp['lang']= final
yelp = yelp[yelp['lang']=='en']

yelp.lang.value_counts()

0
This row throws an error: !


en    99579
Name: lang, dtype: int64

In [ ]:
yelp = yelp.drop(['lang','useful','stars','review_id','user_id','business_id','cool','funny','date'],axis=1)

In [33]:
yelp.to_csv('../data/yelp-dataset/yelp2.csv',index=False)

## Data cleaning for coursera dataset

In [23]:
DF = pd.read_csv('../data/coursera/reviews_coursera.csv')

DF = DF.rename(columns={'Review':'text','Label':'stars'})

DF_drop = DF[DF['stars']!= 3.0]

DF_drop['stars'].value_counts()

5    79173
4    18054
1     2469
2     2251
Name: stars, dtype: int64

In [24]:
DF_drop = DF_drop.drop(['Id'],axis=1)

In [25]:
DF_drop

,text,stars
0,good and interesting,5
1,"This class is very helpful to me. Currently, I...",5
2,like!Prof and TAs are helpful and the discussi...,5
3,Easy to follow and includes a lot basic and im...,5
4,Really nice teacher!I could got the point eazl...,4
...,...,...
107013,Trendy topic with talks from expertises in the...,4
107014,"Wonderful! Simple and clear language, good ins...",5
107015,an interesting and fun course. thanks. dr quincy,5
107016,"very broad perspective, up to date information...",4


In [26]:
positive = []

for i in DF_drop.stars:
    if i < 3.0:
        positive.append(0)
    else:
        positive.append(1)
DF_drop['positive'] = positive

DF_pos = DF_drop[DF_drop['positive']==1]
DF_neg = DF_drop[DF_drop['positive']==0]

DF_pos_sample = DF_pos.sample(n=4000,random_state=123)
DF_neg_sample = DF_neg.sample(n=4000,random_state=123)

DF_pos_sample['stars'].value_counts()

5    3247
4     753
Name: stars, dtype: int64

In [27]:
DF_neg_sample['stars'].value_counts()

1    2081
2    1919
Name: stars, dtype: int64

In [28]:
coursera = pd.concat([DF_pos_sample,DF_neg_sample])

In [30]:
coursera = coursera.drop(['stars'],axis=1)

coursera = coursera.reset_index(drop=True)

In [31]:
coursera = coursera.rename(columns={'Review':'text','positive':'pos_class'})
coursera

,text,pos_class
0,I thoroughly enjoyed it. The course material c...,1
1,I thoroughly enjoyed doing this course. I have...,1
2,Great and intuitive course.,1
3,Thoroughly enjoyed this course! Problems and a...,1
4,"Than; you very very much, i hope i can join yo...",1
...,...,...
7995,"Interresting, but too easy to understand. Not ...",0
7996,Very simplistic concepts and tactics which are...,0
7997,"This is the worst, most pathetic course I have...",0
7998,Videos are very brief - slide presentations of...,0


In [32]:
coursera.to_csv('../data/coursera_clean.csv')